In [ ]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

In [ ]:
!pip install langchain-community
from langchain_community.llms import LlamaCpp

llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=50,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

In [ ]:
from langchain import PromptTemplate

template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(template = template, input_variables = ["input_prompt"])

In [ ]:
basic_chain = prompt | llm

In [ ]:
basic_chain.invoke(
    {
        "input_prompt":"Hi! My name is Maarten. What is 1 + 1?"
    })

In [ ]:
llm.invoke("Hi! My name is Maarten. What is 1 + 1?")

In [ ]:
from langchain import LLMChain
from langchain import PromptTemplate

template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""

title_prompt = PromptTemplate(template = template, input_variables = ["summary"])
title = LLMChain(llm = llm, prompt=title_prompt, output_key = "title")

In [ ]:
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""

character_prompt = PromptTemplate(template = template, input_variables=['summary', 'title'])
character = LLMChain(llm = llm, prompt = character_prompt, output_key = "character")

In [ ]:
template ="""<s><|user|>
Create a story about {summary} with the title {title}. The main character is: {character}. Only return the story and it cannot be longer than one paragraph
<|assistant|>"""
story_prompt = PromptTemplate(template=template, input_variables=["summary","title","character"])
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [ ]:
llm_chain = title | character | story

In [ ]:
basic_chain.invoke({"input_prompt":"Hi my name is Rupak. What is an apple?"})

In [ ]:
basic_chain.invoke({"input_prompt":"Hey What's my name?"})

In [ ]:
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
template = template,
input_variables = ["input_prompt", "chat_history"])

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)

In [ ]:
llm_chain.invoke({"input_prompt": "Hi! My name is Rupal. What is 1 + 1?"})

In [ ]:
llm_chain.invoke({"input_prompt":"Hi! What is my name?"})

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

llm_chain = LLMChain(
prompt = prompt,
llm=llm,
memory=memory)


In [ ]:
llm_chain.predict(input_prompt="Hi! My name is Maarten and I am 33 years old. What is 1 + 1?")
llm_chain.predict(input_prompt="What is 3 + 3?")

In [ ]:
llm_chain.invoke({"input_prompt":"What is my name?"})

In [ ]:
llm_chain.invoke({"input_prompt":"What is my age?"})

In [ ]:
from langchain import PromptTemplate
summary_prompt_template = """<s><|user|>Summarize the conversations and update with the new lines.

Current Summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""

summary_prompt = PromptTemplate(
input_variables = ["summary", 'new_lines'],
template = summary_prompt_template,
)

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain import LLMChain

memory = ConversationSummaryMemory(
llm = llm,
memory_key = "chat_history",
prompt = summary_prompt)

llm_chain = LLMChain(
prompt = prompt,
llm=llm,
memory = memory)

In [ ]:
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

In [ ]:
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

In [ ]:
memory.load_memory_variables({})

LLM Agents

In [ ]:


openai_llm = ChatOpenAI(api_key="", model_name = 'gpt-3.5-turbo', temperature = 0)

In [ ]:
from langchain_core.prompts import PromptTemplate
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
template = react_template,
input_variables = ["tools", "tool_names", "input", "agent_scratchpad"])

In [ ]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [ ]:
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(openai_llm, tools, prompt)

agent_executor = AgentExecutor(
agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)